### 데이터 전처리
1. 필요없는 컬럼 삭제
2. 라벨을 가장 뒤로 보내기
3. 무한대값 처리
    * 무한대값 있는 컬럼 확인후 컬럼의 최대, 최소 값으로 대체
    * 무한대 값을 최대,최소 값으로 대체하는게 맞나...?
4. 이상값 처리 (윈저라이징 1%)
5. 데이터 값 소수 둘째자리 반올림

파생변수 만드는데 결측치 -> 0 처리
무한대 -> 최대, 최소 => ??

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("결측치처리완료.csv")
df = df.drop(columns=['Unnamed: 0'])

In [3]:
columns_to_remove = [
    '회사명', '거래소코드','총자본순이익률', '매출액증가율',
    '자기자본순이익률', '매출액순이익률', '유동비율', '자기자본구성비율',
    '총자본회전률', '자기자본회전률', '총자본투자효율', '설비투자효율', '순이익증가율'
]

df = df.drop(columns=columns_to_remove)

label_column = df.pop('label')
df['label'] = label_column

In [4]:
def replace_infinity_except_column(df, exclude_column):
    for col in df.columns:
        if col != exclude_column and np.isinf(df[col]).any():
            # 양의 무한대 값을 해당 컬럼의 최대값으로 대체
            max_val = df[col][~np.isinf(df[col])].max()
            df[col].replace(np.inf, max_val, inplace=True)
            
            # 음의 무한대 값을 해당 컬럼의 최소값으로 대체
            min_val = df[col][~np.isinf(df[col])].min()
            df[col].replace(-np.inf, min_val, inplace=True)

# '회계년도' 컬럼을 제외하고 함수 실행
replace_infinity_except_column(df, '회계년도')

# 결과 출력
print(df)            

            회계년도  총자본순이익률_차이  총자본순이익률_비율  총자본순이익률_상위10개_차이  총자본순이익률_상위10개_변동성  \
0     2012-12-01      -10.20    1.827957           -46.709             46.709   
1     2013-12-01       16.30   -1.032953           -21.546             21.546   
2     2014-12-01       -0.24   -0.461538           -32.293             32.293   
3     2015-12-01       -6.65  -23.750000           -56.640             56.640   
4     2016-12-01        8.43   -1.323391           -40.578             40.578   
...          ...         ...         ...               ...                ...   
8622  2015-12-01       -2.40   -0.189573           -40.010             40.010   
8623  2016-12-01      -21.58   -2.103314           -53.958             53.958   
8624  2017-12-01        7.23   -0.638693           -81.108             81.108   
8625  2018-12-01       -5.05    1.234719           -45.142             45.142   
8626  2019-12-01       11.80   -1.291028           -42.441             42.441   

      총자본순이익률_산업평균_차이  총자본순

In [5]:
inf_columns = df.columns[df.isin([np.inf, -np.inf]).any()]
if not inf_columns.empty:
    print("무한대값 또는 -무한대값이 있는 컬럼:")
    for column in inf_columns:
        print(column)
else:
    print("무한대값 또는 -무한대값이 데이터프레임에 없습니다.")

print()

무한대값 또는 -무한대값이 데이터프레임에 없습니다.



In [6]:
def custom_winsorize(series, lower_quantile, upper_quantile):
    quantiles = series.quantile([lower_quantile, upper_quantile])
    return series.clip(lower=quantiles.iloc[0], upper=quantiles.iloc[1])

# 윈저라이징 적용 (회계년도 및 라벨 컬럼 제외)
exclude_columns = ['회계년도', 'label']
for col in df.columns.drop(exclude_columns):
    df[col] = custom_winsorize(df[col], 0.01, 0.99)

# 결과 출력
print(df)

            회계년도  총자본순이익률_차이  총자본순이익률_비율  총자본순이익률_상위10개_차이  총자본순이익률_상위10개_변동성  \
0     2012-12-01      -10.20    1.827957           -46.709             46.709   
1     2013-12-01       16.30   -1.032953           -21.546             21.546   
2     2014-12-01       -0.24   -0.461538           -32.293             32.293   
3     2015-12-01       -6.65  -23.750000           -56.640             56.640   
4     2016-12-01        8.43   -1.323391           -40.578             40.578   
...          ...         ...         ...               ...                ...   
8622  2015-12-01       -2.40   -0.189573           -40.010             40.010   
8623  2016-12-01      -21.58   -2.103314           -53.958             53.958   
8624  2017-12-01        7.23   -0.638693           -81.108             81.108   
8625  2018-12-01       -5.05    1.234719           -45.142             45.142   
8626  2019-12-01       11.80   -1.291028           -42.441             42.441   

      총자본순이익률_산업평균_차이  총자본순

In [7]:
df = df.round(2)

In [8]:
df.to_csv("데이터전처리완료.csv")

In [9]:
df.describe()

,총자본순이익률_차이,총자본순이익률_비율,총자본순이익률_상위10개_차이,총자본순이익률_상위10개_변동성,총자본순이익률_산업평균_차이,총자본순이익률_산업평균_변동성,매출액증가율_차이,매출액증가율_비율,매출액증가율_상위10개_차이,매출액증가율_상위10개_변동성,...,설비투자효율_상위10개_변동성,설비투자효율_산업평균_차이,설비투자효율_산업평균_변동성,순이익증가율_차이,순이익증가율_비율,순이익증가율_상위10개_차이,순이익증가율_상위10개_변동성,순이익증가율_산업평균_차이,순이익증가율_산업평균_변동성,label
count,8627.000000,8627.000000,8627.000000,8627.000000,8627.000000,8627.000000,8627.000000,8627.000000,8627.000000,8627.000000,...,8627.000000,8627.000000,8627.000000,8627.000000,8627.000000,8627.000000,8627.000000,8627.000000,8627.000000,8627.000000
mean,-0.457413,-0.610347,-44.984392,45.124233,0.127088,9.332911,-0.987933,-0.590078,-2773.717865,2777.472147,...,74888.459575,8779.235410,10077.941914,1.761323,340.559339,-67019.648516,67127.981711,-793.193554,857.703789,0.016112
std,12.204838,4.446365,20.634578,20.505647,13.546837,10.489130,50.441909,6.743663,5522.958500,5524.207387,...,78226.780473,22772.244354,22303.713549,172.076152,2140.180688,187244.949901,187344.806220,2301.871097,2303.172649,0.125914
min,-45.640000,-28.080000,-110.000000,7.650000,-58.800000,0.140000,-196.410000,-32.720000,-16396.420000,71.490000,...,5280.930000,-3087.040000,8.820000,-784.470000,-2001.350000,-646159.530000,539.330000,-7973.110000,3.160000,0.000000
25%,-4.010000,-0.930000,-54.820000,30.640000,-3.120000,2.960000,-17.095000,-1.590000,-821.155000,422.320000,...,11084.750000,-624.845000,186.200000,-15.375000,-1.050000,-3940.450000,2377.790000,-124.785000,40.730000,0.000000
50%,-0.030000,-0.200000,-42.070000,42.110000,2.910000,6.210000,0.000000,-0.670000,-505.650000,506.180000,...,25198.890000,-140.510000,551.710000,0.000000,0.000000,-3811.680000,3811.680000,-55.820000,58.880000,0.000000
75%,2.765000,0.230000,-30.605000,54.880000,7.370000,11.410000,13.545000,0.060000,-421.570000,821.345000,...,121055.455000,250.265000,2503.685000,11.290000,0.000000,-2377.790000,3941.730000,-33.280000,205.175000,0.000000
max,47.160000,14.470000,-3.720000,110.210000,26.430000,62.280000,214.450000,36.490000,-41.000000,16396.710000,...,225422.820000,65207.970000,65229.430000,876.910000,11706.000000,-55.990000,646159.590000,823.220000,7975.000000,1.000000
